In [1]:
from mnemonic import Mnemonic
from fastecdsa import keys, curve
import binascii
import hashlib
import hmac



In [2]:
"""here i create a class to store data in a tree like structure"""

class TreeNode():
    
    def __init__(self,parent):
        self.parent=parent
        self.pvt_key=None
        self.pub_key=None
        self.chain_code =None
        self.children = []
    

In [3]:
"""Here I define the function needed to generate the seed which is going to be used later """

def generate_seed(language,passphrase,strenght):
   mnemo = Mnemonic(language)
   words = mnemo.generate(strength=strenght)
   seed = mnemo.to_seed(words, passphrase=passphrase)
   print("Entropy=" + str( binascii.hexlify(mnemo.to_entropy(words)))) #prints the entropy in hex
   print("Words= " + words)   
   print("Seed = "+str(binascii.hexlify(seed))) #prints the seed in hex 
   return seed
   


In [4]:
seed=generate_seed("english","test",128)

Entropy=b'f7ae6740de303094752471d6685c9927'
Words= waste industry south round adapt enforce stand mixed stool drill chaos example
Seed = b'1e2453715fd9f1fd6758f8f746be5875de5fe04638277bc1ef5b9c884466255a1b5517e8fffd9075756aada2d28b9675db9bfceee3a6da0509ec50224d33781f'


In [5]:
"""In this section I derive from the seed the master private key and the master chain code, the private key is the one used to have control over your bank account represented by the public key. The bitcoin address is generated/corresponds from the public key"""

def get_master_info(seed):
    master=TreeNode(None)
    master_hmac= hmac.new(seed,None,hashlib.sha512).hexdigest()
    master_pvt_key=master_hmac[:64]
    master.pvt_key=master_pvt_key
    master_chain_code=master_hmac[64:]
    master.chain_code=master_chain_code
    pvt=int(master_pvt_key, 16)
    master_pub_key = keys.get_public_key(pvt,curve.secp256k1)
    pub_key= hex(master_pub_key.y)[2:4]+hex(master_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
    master.pub_key=pub_key
    return master


In [6]:
master_info=get_master_info(seed)
master_info.pub_key


'83f3c2ceafdb81dee4c2f40990501340b6f3927ac8033411f435dc68f2d3b9b18f'

In [7]:
"""In this section I define a function to derive child addresses using hardened derivation"""

def create_hardened_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pvt_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)
            

In [8]:
"""in this section I create a children from the public key of the master"""

def create_children(parent,index,num): #the index should be passed as a 32 bit hex
  
    for i in range(0,num): 
        child=TreeNode(parent.pvt_key)
        input_bytearray=str.encode(parent.pub_key)+str.encode(parent.chain_code)+str.encode(hex(int(index, 16)+i)[2:])
        child_info=hmac.new(input_bytearray,None,hashlib.sha512).hexdigest()
        child.pvt_key=child_info[:64]
        child.chain_code=child_info[64:]
        pvt=int(child.pvt_key, 16)
        child_pub_key=keys.get_public_key(pvt,curve.secp256k1)
        pub_key= hex(child_pub_key.y)[2:4]+hex(child_pub_key.x)[2:] #followed the guide https://en.bitcoin.it/wiki/Technical_background_of_version_1_Bitcoin_addresses#How_to_create_Bitcoin_Address
        child.pub_key=pub_key
        parent.children.append(child)

In [9]:
create_children(master_info,"00000000",2)
create_hardened_children(master_info,"80000000",1)

for i in master_info.children:
        print("Private Key " + i.pvt_key)
        print("Public Key " + i.pub_key)
        print("Chain Code " + i.chain_code)
        print("-------------------------------------------------------------------------")
        

Private Key 3b1396255700474b63cfc368ab6c4220e490d22237e831638530e0e6f4481fca
Public Key 831ee5bbe20834154ffa0330ab94943aa4e7a63e3bbabf971bed7a2a4789de19c2
Chain Code d21d1a78d9380d47e869db2bb601b80c4de7b148f0009c951912e11f0e6d5d83
-------------------------------------------------------------------------
Private Key 2b2b4af504fa65612748c90d154c0e24cf2184e356036588250513cd44a298dd
Public Key a37286b22e5fa114206efa848b31f1b4003005f1c60a18c2de4293909c2fc73e52
Chain Code 38737ea6afa3e71a8b7d9b00a3860331900ae4293c165162791dd575569998d7
-------------------------------------------------------------------------
Private Key 9ff2f619286b9711188458434e77da48b67c26751699e86d0a9f81d5ff384ead
Public Key 11af8aab9f75e9a12c2cf30169cb17ed5d6c6b835661f4b5e1fc8b80c22b9afbdf
Chain Code cfaf872c8404feb4bf440b3ae879eb930fd01cb238876b9c358a5221ba38bb89
-------------------------------------------------------------------------


In [10]:
"""Lets try to create some grandkids"""

for i in master_info.children:
    create_hardened_children(i,"80000000",1)
    create_children(i,"00000000",2)
    for j in i.children:
        print("Private Key " + j.pvt_key)
        print("Public Key " + j.pub_key)
        print("Chain Code " + j.chain_code)
        print("-------------------------------------------------------------------------")
        




Private Key 7f619d12cd3189a3774a4fe7394a7db3926f843b9dca94344b966095598853ea
Public Key f6297ab1fe6ea5f51a4cffe5f6790d7d888ff30c6f33d33f800f953b17de9747fc
Chain Code c726eadc19691444f4835b066f219dd5ccfc333374c308db3944cbd2512d86e7
-------------------------------------------------------------------------
Private Key ac8e60a185ffe3d3624c87b1b1fce776e036e2f9197b4d50647548893dadb1b4
Public Key 9b84fd1ec73a54527354e57648c2056d1e15a568c1350dd90ef5b6b0871af7f271
Chain Code 95c2b9654ac04029e604dcb1a5bdcb0f868df10ff5da9f5c2a627348d0677798
-------------------------------------------------------------------------
Private Key 11995120e7da1f7e12734f547c48f97973c189d280e7d256be022722f2b8ad6b
Public Key 81f21c53cb30f2b674e8eeebcebbb7fdf93faa65392a178cadededad9b6564957b
Chain Code 16770d6fd84b6481e6d11fe70707d19de2eeb3e8e9cf4c3baf903d9aededc9d6
-------------------------------------------------------------------------
Private Key 7de5048a8f960c222c818cb8b6faac5f44b213542b2a8392ca076459297b5f06
Public K